In [ ]:
from datasketch import LeanMinHash, MinHash, MinHashLSH
from nltk.util import ngrams

from yadbil.data.telegram import TelegramDataProcessor

In [ ]:
INPUT_DATA = "data/result.json"

GRAPH_FILE_PATH = "data/filtered_graph.graphml"
POSTS_FILE_PATH = "data/posts.json"
POSTS_VIEW_FILE_PATH = "data/posts_view.json"

Data processing

In [ ]:
# TODO: filter on len(text_no_link)<30
# TODO: feature for ui, to show posts close in timeline for the channel
tg_data_proc = TelegramDataProcessor(INPUT_DATA).process_posts()
posts = tg_data_proc.posts
posts_view = tg_data_proc.posts_view

In [ ]:
words_posts = [set(x["stemmed_words"]) for x in posts]

In [ ]:
def apply_char_ngrams(words, n):
    ngram_set = set()
    for word in words:
        if len(word) < n:
            ngram_set.add(word)
            continue
        char_ngrams = ["".join(gram) for gram in ngrams(word, n)]
        ngram_set.update(char_ngrams)
    return ngram_set

In [ ]:
threegrams_posts = [apply_char_ngrams(x, 4) for x in words_posts]

In [ ]:
def minhash_from_set(set_, num_perm=128):
    m = MinHash(num_perm=num_perm)
    m.update_batch([x.encode("utf8") for x in set_])
    return LeanMinHash(m)


# TODO: add proper id
def minhashlsh_from_iterable(minhashes, threshold=0.5, num_perm=128):
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for i, x in enumerate(minhashes):
        lsh.insert(f"{i}", x)
    return lsh

In [ ]:
minhashes = [minhash_from_set(x, num_perm=256) for x in threegrams_posts]

# Create LSH index
lsh = minhashlsh_from_iterable(minhashes=minhashes, threshold=0.7, num_perm=256)

# result = lsh.query(minhashes[100])
# print("Approximate neighbours with Jaccard similarity > 0.5", result)

In [ ]:
from tqdm.auto import tqdm


for i, x in tqdm(enumerate(minhashes)):
    result = lsh.query(x)
    if len(result) > 1:
        print(i, "-", result)

In [ ]:
posts_texts = [p["text"] for p in posts]

In [ ]:
print(posts_texts[356])

In [ ]:
print(posts_texts[357])